In [ ]:
import sys, os
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), '../')))
from stochss_compute.cloud.ec2 import EC2Cluster, p

In [ ]:
cluster = EC2Cluster()

In [ ]:
vpc_cidrBlock = '172.31.0.0/16'
vpc_tag = [
    {
        'ResourceType': 'vpc',
        'Tags': [
            {
                'Key': 'Name',
                'Value': 'sssc-vpc'
            }
        ]
    }
]
vpc_response = cluster.client.create_vpc(CidrBlock=vpc_cidrBlock, TagSpecifications=vpc_tag)
vpc_id = vpc_response['Vpc']['VpcId']
cluster.client.modify_vpc_attribute( VpcId = vpc_id , EnableDnsSupport = { 'Value': True } )
cluster.client.modify_vpc_attribute( VpcId = vpc_id , EnableDnsHostnames = { 'Value': True } )

In [ ]:
vpc = cluster.resources.Vpc(vpc_id)
for igw in vpc.internet_gateways.all():
    print(igw)


In [ ]:
igw_response = cluster.client.create_internet_gateway()
igw_id = igw_response['InternetGateway']['InternetGatewayId']
vpc.attach_internet_gateway(InternetGatewayId=igw_id)

In [ ]:
for rtb in vpc.route_tables.all():
    if rtb.associations_attribute[0]['Main'] == True:
        rtb_id = rtb.route_table_id
rtb_id

In [ ]:
cluster.client.create_route(RouteTableId=rtb_id, GatewayId=igw_id, DestinationCidrBlock='0.0.0.0/0')

In [ ]:
cluster.launch_single_node_cluster()

In [ ]:
vpcId = 'vpc-0aaae1dae8fe0eac7'
vpc = cluster.resources.Vpc(vpcId)

In [ ]:
for subnet in vpc.subnets.all():
    print(subnet)
    subnet.delete()

In [ ]:
for sg in vpc.security_groups.all():
    sg.delete()

In [ ]:
for igw in vpc.internet_gateways.all():
    print(igw)
    igw.detach_from_vpc(VpcId=vpc.vpc_id)
    igw.delete()

In [ ]:
for rtb in vpc.route_tables.all():
    print(rtb)
    for assoc in rtb.associations:
        print(assoc.association_state)
        assoc.delete()
    # rtb.delete()
    # rtb.delete()

In [ ]:
head = cluster.instances[0]

In [ ]:
# cluster.create_root_key()
vpcId = cluster.create_default_vpc()
print(f'vpcId: {vpcId}')


In [ ]:
subnetId = cluster.create_default_subnet(vpcId)
print(f'subnetId: {subnetId}')


In [ ]:
sgId = cluster.create_default_security_group(vpcId)
print(f'sgId: {sgId}')


In [ ]:
head = cluster.launch_instances(subnetId="", securityGroupId="")
print(f'head: {head}')


In [ ]:
head.authorize_SSH()

In [ ]:
cluster.terminate_all_instances()